##Imports

In [ ]:
import sklearn
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_URL = "drive/MyDrive/Citadel Women's Datathon/code"

In [ ]:
zcta_census_tract_df = pd.read_csv(f"{BASE_URL}/census_code_mappers/zcta_census_tract.csv")

In [ ]:
zcta_census_tract_df.head()

,ZCTA10,state_fip,county_fip,tract,geoid
0,601,72,1,956300,72001956300
1,601,72,1,956400,72001956400
2,601,72,1,956500,72001956500
3,601,72,1,956600,72001956600
4,601,72,1,956700,72001956700


In [ ]:
percent_cover_tracts_no_buffer_df = pd.read_csv(f"{BASE_URL}/greenspace_data/percent_cover_tracts_no_buffer.txt")
percent_cover_tracts_no_buffer_df.to_csv
percent_cover_tracts_no_buffer_df.head()

,GEOID,STATEFP,COUNTYFP,pc_park
0,1055010700,1,55,4.423
1,1055001300,1,55,3.706
2,1055000900,1,55,1.037
3,1055001700,1,55,0.510
4,1055010501,1,55,9.279


In [ ]:
MH_CLD_2020_df = pd.read_csv(f"{BASE_URL}/health_datasets/MH-CLD_2020.csv")
MH_CLD_2020_df.head()

,YEAR,AGE,EDUC,ETHNIC,RACE,GENDER,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,...,ODDFLG,PDDFLG,PERSONFLG,SCHIZOFLG,ALCSUBFLG,OTHERDISFLG,STATEFIP,DIVISION,REGION,CASEID
0,2020,14,-9,4,3,1,1,1,1,2,...,0,0,0,1,0,0,1,6,3,20200000001
1,2020,1,2,4,1,1,2,1,2,2,...,0,0,0,0,0,1,1,6,3,20200000002
2,2020,6,-9,4,4,2,2,1,1,2,...,0,0,0,0,0,0,1,6,3,20200000003
3,2020,14,-9,4,3,2,1,1,2,2,...,0,0,0,1,0,0,1,6,3,20200000004
4,2020,13,-9,4,3,2,1,1,1,2,...,0,0,1,1,0,1,1,6,3,20200000005


In [ ]:
tax_returns_df = pd.read_csv(f"{BASE_URL}/20zpallagi.csv")
tax_returns_df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,1,785000.0,519980.0,85690.0,165290.0,724170.0,22560.0,...,0.0,0.0,57720.0,46577.0,674840.0,1827202.0,672200.0,1818867.0,2900.0,6089.0
1,1,AL,0,2,554310.0,270870.0,121420.0,146470.0,515150.0,13260.0,...,0.0,0.0,81770.0,112540.0,470410.0,1445383.0,466960.0,1432458.0,4660.0,11648.0
2,1,AL,0,3,290630.0,113280.0,124770.0,44570.0,269700.0,6420.0,...,0.0,0.0,70360.0,144380.0,220710.0,626662.0,216530.0,610170.0,5760.0,16235.0
3,1,AL,0,4,181010.0,42010.0,120820.0,14410.0,168830.0,2570.0,...,0.0,0.0,49500.0,135429.0,130670.0,437179.0,126790.0,419324.0,3730.0,14903.0
4,1,AL,0,5,269080.0,31310.0,224330.0,8270.0,252360.0,3250.0,...,100.0,20.0,103250.0,470206.0,165650.0,724529.0,156910.0,642895.0,11280.0,80064.0


In [ ]:
# We only need the first four columns for tax_returns_df: STATEFIPS, STATE, ZIPCODE, AGI_STUB
tax_returns_df.shape

(166452, 165)

In [ ]:
tax_returns_df.columns

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'ELF', 'CPREP',
       ...
       'N85300', 'A85300', 'N11901', 'A11901', 'N11900', 'A11900', 'N11902',
       'A11902', 'N12000', 'A12000'],
      dtype='object', length=165)

## Transform tax returns


In [ ]:
tax_returns_df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,1,785000.0,519980.0,85690.0,165290.0,724170.0,22560.0,...,0.0,0.0,57720.0,46577.0,674840.0,1827202.0,672200.0,1818867.0,2900.0,6089.0
1,1,AL,0,2,554310.0,270870.0,121420.0,146470.0,515150.0,13260.0,...,0.0,0.0,81770.0,112540.0,470410.0,1445383.0,466960.0,1432458.0,4660.0,11648.0
2,1,AL,0,3,290630.0,113280.0,124770.0,44570.0,269700.0,6420.0,...,0.0,0.0,70360.0,144380.0,220710.0,626662.0,216530.0,610170.0,5760.0,16235.0
3,1,AL,0,4,181010.0,42010.0,120820.0,14410.0,168830.0,2570.0,...,0.0,0.0,49500.0,135429.0,130670.0,437179.0,126790.0,419324.0,3730.0,14903.0
4,1,AL,0,5,269080.0,31310.0,224330.0,8270.0,252360.0,3250.0,...,100.0,20.0,103250.0,470206.0,165650.0,724529.0,156910.0,642895.0,11280.0,80064.0


In [ ]:
def get_median_income_class_for_zipcode(zipcode):
  if zipcode == 0:
    # 0 corresponds to the cumulative returns for an entire state but we're checking 
    # on the granularity of a zipcode 
    return None

  row_range = np.where((tax_returns_df["zipcode"] == zipcode))

  agi_stub_to_num = {}
  for i in range(min(row_range[0]), max(row_range[0]) + 1):
    agi_stub = tax_returns_df.at[i, 'agi_stub']
    n1 = tax_returns_df.at[i, 'N1']
    agi_stub_to_num[agi_stub] = n1


  median = sum(agi_stub_to_num.values()) / 2
  median_income_class = None
  current_sum = 0
  for key in agi_stub_to_num:
    if current_sum >= median:
      median_income_class = key
      break
    current_sum += agi_stub_to_num[key]
  if not median_income_class:
    median_income_class = max(agi_stub_to_num.keys())
  
  return median_income_class

In [ ]:
# Get all zipcodes
all_zips = tax_returns_df["zipcode"].unique()
all_zips = all_zips[all_zips != 0]
all_zips

array([35004, 35005, 35006, ..., 83127, 83128, 83414])

In [ ]:
tax_returns_median_df_dict = {"ZIPCODE": [], "MEDIAN_INCOME": []}
for zip in all_zips:
  income = get_median_income_class_for_zipcode(zip)
  tax_returns_median_df_dict["ZIPCODE"].append(zip)
  tax_returns_median_df_dict["MEDIAN_INCOME"].append(income)

tax_returns_median_df = pd.DataFrame(tax_returns_median_df_dict, columns=['ZIPCODE', 'MEDIAN_INCOME'])

In [ ]:
tax_returns_median_df.tail()

,ZIPCODE,MEDIAN_INCOME
27638,83122,3
27639,83126,4
27640,83127,4
27641,83128,4
27642,83414,5


In [ ]:
tax_returns_median_df.shape

(27643, 2)

In [ ]:
# Export tax_returns_median
tax_returns_median_df.to_csv(f"{BASE_URL}/tax_returns_median.csv")

In [ ]:
zip_tract_df = pd.read_excel(f'{BASE_URL}/ZIP_TRACT_122021.xlsx', index_col=0) 

In [ ]:
zip_tract_df.head()

,tract,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
zip,,,,,,,
683,72023830102,SAN GERMAN,PR,0.000791,0.001116,0.000000,0.000800
683,72125840700,SAN GERMAN,PR,0.186219,0.370536,0.381643,0.201179
683,72125840400,SAN GERMAN,PR,0.300451,0.187500,0.115942,0.290308
683,72125840600,SAN GERMAN,PR,0.095325,0.007812,0.000000,0.088184
683,72121960300,SAN GERMAN,PR,0.042402,0.002232,0.019324,0.039435


In [ ]:
col_names = ["zipcode", "pc_park", "med_income"]
zip_pcpark_income_df = pd.DataFrame(columns=col_names)

In [ ]:
zip_pcpark_income_dict = {"zipcode":[], "pc_park":[], "med_income":[]}
for index, row in tax_returns_median_df.iterrows():
  zip_pcpark_income_dict["zipcode"] = row["ZIPCODE"]  

In [ ]:
zip_pcpark_income_dict

{'zipcode': 83414, 'pc_park': [], 'med_income': []}

In [ ]:
def get_zip_to_pc_park():
  d = {}
  num_errors = 0
  for i in range(1, len(percent_cover_tracts_no_buffer_df) - 1):
    # print(i)
    pc_park = percent_cover_tracts_no_buffer_df["pc_park"][i]
    geoid = percent_cover_tracts_no_buffer_df["GEOID"][i]
    idx = np.where((zip_tract_df["tract"] == geoid))

    if not idx:
      continue

    try:
      zipcode = zip_tract_df.iloc[idx].index[0]
    except:
      num_errors += 1
      continue
      # return
    if zipcode not in d:
      d[zipcode] = [pc_park]
    else:
      d[zipcode].append(pc_park)

  output = {"ZIPCODE": [], "PC_PARK": []}
  for key in d:
    output['ZIPCODE'].append(key)
    output['PC_PARK'].append(sum(d[key])/len(d[key]))
  return output

In [ ]:
zip_pc_park_dict = get_zip_to_pc_park()

In [ ]:
zip_pc_park_df = pd.DataFrame(zip_pc_park_dict)

In [ ]:
tax_returns_median_df.head()

,ZIPCODE,MEDIAN_INCOME
0,35004,3
1,35005,3
2,35006,3
3,35007,3
4,35010,3


In [ ]:
zip_pc_park_df.head()

,ZIPCODE,PC_PARK
0,35903,3.411571
1,35904,2.707100
2,35906,9.741500
3,35972,0.000000
4,35954,0.154000


In [ ]:
output_df = pd.merge(zip_pc_park_df,tax_returns_median_df,on='ZIPCODE')

In [ ]:
output_df.head()

,ZIPCODE,PC_PARK,MEDIAN_INCOME
0,35903,3.411571,3
1,35904,2.707100,3
2,35906,9.741500,3
3,35972,0.000000,3
4,35954,0.154000,3


In [ ]:
output_df.shape

(15645, 3)

In [ ]:
output_df.to_csv(f"{BASE_URL}/output_df.csv")

In [ ]:
output_df.head()

,ZIPCODE,PC_PARK,MEDIAN_INCOME
0,35903,3.411571,3
1,35904,2.707100,3
2,35906,9.741500,3
3,35972,0.000000,3
4,35954,0.154000,3
